In [ ]:
# Importat librerías necesarias
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Definir la clase EuclideanDisTracker para medir la distancia entre los objetos
class EuclideanDistTracker:
    def __init__(self):
        # Almacena los centros de cada objeto detectado
        self.center_points = {}
        # Contador para asignar IDs únicos
        self.id_count = 0

    def update(self, objects_rect):
        """
        Recibe una lista de bounding boxes [x, y, w, h] y devuelve
        la misma lista con un ID asignado a cada objeto: [x, y, w, h, id].
        """
        objects_bbs_ids = []

        # Para cada rectángulo, calcular su centro
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Buscar si ya tenemos un objeto cercano (distancia < 25 píxeles)
            same_object_detected = False
            for object_id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])
                if dist < 25:
                    # Actualizar centro y conservar el mismo ID
                    self.center_points[object_id] = (cx, cy)
                    objects_bbs_ids.append([x, y, w, h, object_id])
                    same_object_detected = True
                    break

            # Si no se encontró ninguno cercano, es un objeto nuevo
            if not same_object_detected:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Limpiar IDs que ya no están presentes
        new_center_points = {}
        for _, _, _, _, object_id in objects_bbs_ids:
            new_center_points[object_id] = self.center_points[object_id]
        self.center_points = new_center_points.copy()

        return objects_bbs_ids

In [ ]:
# Configuración de captura de video y detector de fondo
tracker = EuclideanDistTracker()
cap = cv2.VideoCapture("highway.mp4")

# Utilizaremos un detector de fondo tipo MOG2 para extraer el fondo del video
object_detector = cv2.createrBackgroundSubtractorMOG2(history=100, varThreshold=40)


In [ ]:
# Aqui es donde comenzaremos la detección y el seguimiento de los objetos con la cámara
while True:
    ret, frame = cap.read()
    if not ret:
        break # Terminamos si no hay frames que procesar/leer

    # A diferencia de la imagen, ahora buscaremos objetos en un área en especifico o zona de interes (ROI)
    roi = frame[340:720, 500:800]

    # Detección de objetos con sustracción de fondo
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    detections = []
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 100:
            x,y,w,h = cv2.boundingRect(cnt)
            detections.append([x,y,w,h])

    